In [15]:
import json
import pandas as pd
import numpy as np
import torch
from torch import nn
from pprint import pprint
import jsonlines
import os
import re
from tqdm.notebook import tqdm
import pickle

from typing import *

In [2]:
train_path = 'data/train.jsonl'
dev_path = 'data/dev.jsonl'

# Utils

In [9]:
def save_dictionary(dictionary, path):
    with open(path, 'wb') as f:
        pickle.dump(dictionary, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_dictionary(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# Create word embedding with GloVe

In [5]:
def sentence2vector(sentence: str) -> Optional[torch.Tensor]:
    sentences_word_vector = [word_vectors[w] for w in sentence.split(' ') if w in word_vectors]
    
    if len(sentences_word_vector) == 0:
        return None

    sentences_word_vector = torch.stack(sentences_word_vector)  # tensor shape: (#words X #features)
    return torch.mean(sentences_word_vector, dim=0)

In [4]:
def cosine_similarity(v1: torch.Tensor, v2: torch.Tensor) -> float:
    num = torch.sum(v1 * v2)
    den = torch.linalg.norm(v1) * torch.linalg.norm(v2)
    return (num / den).item()

In [3]:
word_vectors = dict()
n_words = 400_000
with open('embeddings/glove.6B.300d.txt') as f:
    for i, line in tqdm(enumerate(f), total=n_words):

        word, *vector = line.strip().split(' ')
        vector = torch.tensor([float(c) for c in vector])
        
        word_vectors[word] = vector

  0%|          | 0/400000 [00:00<?, ?it/s]

# Dataset class using GloVe

In [29]:
class WiCDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path: str):
        self.data = []
        self.create_dataset(dataset_path)
    
    def create_dataset(self, dataset_path: str) -> None:
        with jsonlines.open(dataset_path, 'r') as f:
            for i, line in enumerate(f.iter()):
                # s1 = sentence2vector(line['sentence1'])
                # s2 = sentence2vector(line['sentence2'])
                sentence = f"{line['sentence1']} {line['sentence2']}"
                sentence_vector = sentence2vector(sentence)
                
                label = 1 if line['label'] == 'True' else 0
                self.data.append((sentence_vector, label))


    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.data[idx]

# Model Class

In [16]:
class MLP(nn.Module):
    def __init__(
        self,
        n_features: int,
        num_layers: int,
        hidden_dim: int,
        activation: Callable[[torch.Tensor], torch.Tensor],
    ) -> None:
        super().__init__()

        self.first_layer = nn.Linear(in_features=n_features, out_features=hidden_dim)

        self.layers = (
            nn.ModuleList()
        )

        for i in range(num_layers):
            self.layers.append(
                nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
            )
        self.activation = activation

        self.last_layer = nn.Linear(in_features=hidden_dim, out_features=1)

    def forward(self, meshgrid: torch.Tensor) -> torch.Tensor:
        """
        Applies transformations to each (x, y) independently

        :param meshgrid: tensor of dimensions [..., 2], where ... means any number of dims
        """
        out = meshgrid

        out = self.first_layer(
            out
        )  # First linear layer, transforms the hidden dimensions from `n_features` (embedding dimension) to `hidden_dim`
        for layer in self.layers:  # Apply `k` (linear, activation) layer
            out = layer(out)
            out = self.activation(out)
        out = self.last_layer(
            out
        )  # Last linear layer to bring the `hiddem_dim` features to a binary space (`True`/`False`)

        return out.squeeze(-1)


# Instantiate model

In [35]:
model = MLP(n_features=300,
            num_layers=3,
            hidden_dim=100,
            activation=torch.nn.functional.relu)
model.parameters  # Explore the parameters, i.e. the trainable tensors that require a grad in this model

<bound method Module.parameters of MLP(
  (first_layer): Linear(in_features=300, out_features=100, bias=True)
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
  )
  (last_layer): Linear(in_features=100, out_features=1, bias=True)
)>

# Instantiate dataset and dataloader

In [31]:
train_dataset = WiCDataset(train_path)

In [32]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

## Test model

In [33]:
x, y = next(iter(train_loader))

In [36]:
model(x), y

(tensor([0.0918, 0.0936, 0.0903, 0.0937, 0.0951, 0.0913, 0.0925, 0.0924, 0.0902,
         0.0936, 0.0944, 0.0937, 0.0935, 0.0925, 0.0946, 0.0908],
        grad_fn=<SqueezeBackward1>),
 tensor([1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1]))